In [1]:
!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 8.7 MB/s eta 0:00:00


In [2]:
!head -5 forestfireinput.csv

7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0,0
7,4,oct,tue,90.6,35.4,669.1,6.7,18,33,0.9,0,0
7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0,0
8,6,mar,fri,91.7,33.3,77.5,9,8.3,97,4,0.2,0
8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0,0


In [12]:
%%file forestfireavg.py

from mrjob.job import MRJob

class ForestFireAvarageTemp(MRJob):
  def mapper(self, _, value):
    fields = value.split(",")
    if len(fields) == 13:
      try:
        temperature = float(fields[8])
        yield "Temp", [temperature, 1] #chave comum e valor composto (num,1)
      except ValueError:
        pass

  def reducer(self, key, value):
    sum_values = 0
    count = 0
    for v in value:
      sum_values += v[0]
      count += v[1]
    avg = sum_values / count
    yield "Average", avg

if __name__ == '__main__':
  ForestFireAvarageTemp.run()

Overwriting forestfireavg.py


In [13]:
!rm -rf output
!python forestfireavg.py forestfireinput.csv --output-dir=output

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/forestfireavg.root.20250905.140135.523413
job output is in output
Removing temp directory /tmp/forestfireavg.root.20250905.140135.523413...


#Média total e média por mês

In [18]:
%%file forestfireavg_month.py

from mrjob.job import MRJob

class ForestFireAvarageTempMonth(MRJob):
    def mapper(self, _, value):
        fields = value.split(",")
        if len(fields) == 13:
            try:
                temperature = float(fields[8])
                month = fields[2]
                yield "AverageAll", (temperature, 1) # Key for overall average
                yield f"{month}", (temperature, 1) # Key for monthly average
            except ValueError:
                pass

    def reducer(self, key, values):
        sum_values = 0
        count = 0
        for value in values:
            sum_values += value[0]
            count += value[1]
        if count > 0:
            avg = sum_values / count
            yield key, avg

if __name__ == '__main__':
    ForestFireAvarageTempMonth.run()

Overwriting forestfireavg_month.py


In [19]:
!rm -rf output2
!python forestfireavg_month.py forestfireinput.csv --output-dir=output2

!cat output2/part-* > output2/Averages

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/forestfireavg_month.root.20250905.140535.769778
job output is in output2
Removing temp directory /tmp/forestfireavg_month.root.20250905.140535.769778...


#Media by mes e dia da semana, quando vento > 2km/h

mes = [2]
dia = [3]
temperatura = [8]
vento = [10]


---



In [22]:
%%file forestfireavg_complex.py

from mrjob.job import MRJob

class ForestFireAvarageTempComplex(MRJob):
    def mapper(self, _, value):
        fields = value.split(",")
        if len(fields) == 13:
            try:
                temperature = float(fields[8])
                month = fields[2]
                day = fields[3]
                wind = float(fields[10])
                if wind > 2:
                  yield f"{month}, {day}", (temperature, 1)
            except ValueError:
                pass

    def reducer(self, key, values):
        sum_values = 0
        count = 0
        for value in values:
            sum_values += value[0]
            count += value[1]
        if count > 0:
            avg = sum_values / count
            yield key, avg

if __name__ == '__main__':
    ForestFireAvarageTempComplex.run()

Overwriting forestfireavg_complex.py


In [21]:
!rm -rf output3
!python forestfireavg_complex.py forestfireinput.csv --output-dir=output3

!cat output3/part-* > output3/Final

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/forestfireavg_complex.root.20250905.141251.807494
job output is in output3
Removing temp directory /tmp/forestfireavg_complex.root.20250905.141251.807494...


#Media por nivel de vento e nivel de umidade

temperatura = [8]
vento = [10]
umidade = [9]


---



In [26]:
%%file forestfireavg_bylevels.py

from mrjob.job import MRJob

class ForestFireAvarageByLevels(MRJob):
    def mapper(self, _, value):
        fields = value.split(",")
        if len(fields) == 13:
            try:
                temperature = float(fields[8])
                humidity = float(fields[9])
                wind = float(fields[10])
                if wind < 5:
                  if humidity < 30:
                    yield "low wind, low humidity", (temperature, 1)
                  elif humidity >= 30 and humidity <= 60:
                    yield "low wind, medium humidity", (temperature, 1)
                  else:
                    yield "low wind, high humidity", (temperature, 1)
                elif wind >= 5 and wind <= 10:
                  if humidity < 30:
                    yield "medium wind, low humidity", (temperature, 1)
                  elif humidity >= 30 and humidity <= 60:
                    yield "medium wind, medium humidity", (temperature, 1)
                  else:
                    yield "medium wind, high humidity", (temperature, 1)
                else:
                  if humidity < 30:
                    yield "high wind, low humidity", (temperature, 1)
                  elif humidity >= 30 and humidity <= 60:
                    yield "high wind, medium humidity", (temperature, 1)
                  else:
                    yield "high wind, high humidity", (temperature, 1)
            except ValueError:
                pass

    def reducer(self, key, values):
        sum_values = 0
        count = 0
        for value in values:
            sum_values += value[0]
            count += value[1]
        if count > 0:
            avg = sum_values / count
            yield key, avg

if __name__ == '__main__':
    ForestFireAvarageByLevels.run()

Overwriting forestfireavg_bylevels.py


In [28]:
!rm -rf output4
!python forestfireavg_bylevels.py forestfireinput.csv --output-dir=output4

!cat output4/part-* > output4/Final

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/forestfireavg_bylevels.root.20250905.143601.218403
job output is in output4
Removing temp directory /tmp/forestfireavg_bylevels.root.20250905.143601.218403...


In [29]:
%%file forestfireavg_bylevelsV2.py

from mrjob.job import MRJob

class ForestFireAvarageByLevelsV2(MRJob):
    def mapper(self, _, value):
        fields = value.split(",")
        if len(fields) == 13:
          try:
            temperature = float(fields[8])
            humidity = float(fields[9])
            wind = float(fields[10])

            if wind < 5:
              wind_level = "low wind"
            elif wind >= 5 and wind <= 10:
              wind_level = "medium wind"
            else:
              wind_level = "high wind"

            if humidity < 30:
              humidity_level = "low humidity"
            elif humidity >= 30 and humidity <= 60:
              humidity_level = "medium humidity"
            else:
              humidity_level = "high humidity"

            yield f"{wind_level}, {humidity_level}", (temperature, 1)

          except ValueError:
              pass

    def reducer(self, key, values):
        sum_values = 0
        count = 0
        for value in values:
            sum_values += value[0]
            count += value[1]
        if count > 0:
            avg = sum_values / count
            yield key, avg

if __name__ == '__main__':
    ForestFireAvarageByLevelsV2.run()

Writing forestfireavg_bylevelsV2.py


In [30]:
!rm -rf output5
!python forestfireavg_bylevelsV2.py forestfireinput.csv --output-dir=output5

!cat output5/part-* > output5/Final

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/forestfireavg_bylevelsV2.root.20250905.145353.782401
job output is in output5
Removing temp directory /tmp/forestfireavg_bylevelsV2.root.20250905.145353.782401...
